![](https://i.imgur.com/eBRPvWB.png)

# Generowanie sztuki za pomocą RNN i PyTorch w oparciu o litery.

[W tutorialu](https://github.com/spro/practical-pytorch/blob/master/char-rnn-classification/char-rnn-classification.ipynb) użyliśmy RNN, aby sklasyfikować tekst po jednym znaku na raz. Tym razem wygenerujemy tekst po jednym znaku na raz.
```
> python generate.py -n 500

PAOLTREDN:
Let, yil exter shis owrach we so sain, fleas,
Be wast the shall deas, puty sonse my sheete.

BAUFIO:
Sirh carrow out with the knonuot my comest sifard queences
O all a man unterd.

PROMENSJO:
Ay, I to Heron, I sack, againous; bepear, Butch,
An as shalp will of that seal think.

NUKINUS:
And house it to thee word off hee:
And thou charrota the son hange of that shall denthand
For the say hor you are of I folles muth me?
```

Zobacz [Sequence to Sequence Translation tutorial](https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/seq2seq-translation.ipynb) żeby nauczyć się więcej w tym temacie.

## Warto przeczytać

Zakładam, że zainstalowałeś przynajmniej PyTorch, znasz Pythona i rozumiesz Tensory:

* http://pytorch.org/ Instrukcje instalacji
* [Deep Learning with PyTorch: A 60-minute Blitz](https://github.com/pytorch/tutorials/blob/master/Deep%20Learning%20with%20PyTorch.ipynb) na początek z PyTorch (ogólnie)
* [jcjohnson's PyTorch examples](https://github.com/jcjohnson/pytorch-examples) dla szczegółowego przeglądu
* [Introduction to PyTorch for former Torchies](https://github.com/pytorch/tutorials/blob/master/Introduction%20to%20PyTorch%20for%20former%20Torchies.ipynb) jeśli jesteś poprzednim użytkownikiem Lua Torch

Przydałoby się również wiedzieć o RNN i jego działaniu:

* [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) pokazuje kilka przykładów z życia
* [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) dotyczy tylko LSTM, ale także informacji na temat RNN w ogóle

Zobacz także podobne tutoriale z serii:

* [Classifying Names with a Character-Level RNN](https://github.com/spro/practical-pytorch/blob/master/char-rnn-classification/char-rnn-classification.ipynb) używa RNN do klasyfikacji
* [Generating Names with a Conditional Character-Level RNN](https://github.com/spro/practical-pytorch/blob/master/conditional-char-rnn/conditional-char-rnn.ipynb) opiera się na tym modelu, aby dodać kategorię jako dane wejściowe

## Przygotowanie danych

### Podzielenie korpusu na sylaby

In [ ]:
from pathlib import Path
dataset_path = Path('data/rnn_generator'); dataset_path
tmp_path = dataset_path / 'tmp/'
!mkdir -p $tmp_path

In [ ]:
ls -lah $dataset_path/

In [ ]:
fn_corpus_char = dataset_path/'pan_tadeusz.txt'
fn_corpus_syl = dataset_path/'pan_tadeusz.syl1.txt'

In [ ]:
platform_suffixes = {'Linux': 'linux', 'Darwin': 'macos'}
import platform
platform_suffix = platform_suffixes[platform.system()]
stemmer_bin = f'bin/stemmer.{platform_suffix}'

In [ ]:
# !$stemmer_bin -h

In [ ]:
!$stemmer_bin -s 7683 -v -d bin/stemmer2.dic -i $fn_corpus_char -o $fn_corpus_syl

### Załadowanie danych

Plik, którego używamy, jest plikiem tekstowym.

In [ ]:
# fn = 'data/tiny-shakespeare.txt'
# fn = dataset_path / 'mickiewicz.txt'
# fn = dataset_path / 'witkacy_szewcy.txt'
fn = dataset_path / 'pan_tadeusz.txt'

In [ ]:
import string
import random
import re

file = open(fn).read()
file_len = len(file)
print('file_len =', file_len)

# all chars found in file
all_characters = sorted(list(set(file))); print(all_characters[:10])
n_characters = len(all_characters); print(n_characters)

Aby wprowadzić dane z tego dużego ciągu danych, podzielimy je na fragmenty.

In [ ]:
chunk_len = 400

def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

## Budowanie modelu

Ten model przyjmie jako znak wejściowy znak dla kroku $ t _ {- 1} $ i ma wyprowadzić następny znak $ t $. Istnieją trzy warstwy - jedna warstwa liniowa, która koduje znak wejściowy do stanu wewnętrznego, jedna warstwa GRU (która może sama mieć wiele warstw), która działa na tym stanie wewnętrznym i stanie ukrytym, oraz warstwa dekodera, która wyprowadza rozkład prawdopodobieństwa.

### GPU

In [ ]:
import torch

USE_GPU = torch.cuda.is_available(); 
# USE_GPU = False; 

print(f'USE_GPU={USE_GPU}')

def to_gpu(x, *args, **kwargs):
    return x.cuda(*args, **kwargs) if USE_GPU else x

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return Variable(to_gpu(torch.zeros(self.n_layers, 1, self.hidden_size)))

## Wejścia i Wyjścia

Każdy fragment zostanie przekształcony w tensor, w szczególności "LongTensor" (używany do wartości całkowitych), poprzez zapętlenie znaków ciągu i wyszukiwanie indeksu każdego znaku w `all_characters`.

In [ ]:
# Turn string into list of longs
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(to_gpu(tensor))

print(char_tensor('ala ma kota'))

Wreszcie możemy zebrać parę tensorów wejściowych i docelowych do treningu, z losowego kawałka. Wprowadzone zostaną wszystkie znaki * aż do ostatniej *, a celem będą wszystkie znaki * od pierwszego *. Jeśli więc nasz fragment to "abc", dane wejściowe będą odpowiadać "ab", podczas gdy cel to "bc".

In [ ]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

## Ewaluacja

Aby ocenić sieć, będziemy podawać po jednym znaku na raz, wykorzystywać wyjścia sieci jako rozkład prawdopodobieństwa dla następnego znaku i powtarzać. Aby rozpocząć generowanie, przekazujemy ciąg wstępny, aby rozpocząć budowanie stanu ukrytego, z którego następnie generujemy po jednym znaku na raz.

In [ ]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden()
    prime_input = char_tensor(prime_str)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)

    return predicted

## Trenowanie modelu

Pomocnik do wydrukowania upływającego czasu:

In [ ]:
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

Główna funkcja treningowa

In [ ]:
def train(inp, target):
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[c], hidden)
        loss += criterion(output, target[c].expand(1))

    loss.backward()
    decoder_optimizer.step()

    return loss.item() / chunk_len

Następnie definiujemy parametry treningowe i rozpoczynamy trening:

In [ ]:
USE_GPU = True
n_epochs = 3000   # 2000
print_every = 100 # 100
plot_every = 1
hidden_size = 300 # 100
n_layers = 4 # 1
lr = 0.005

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
if USE_GPU:
  decoder.cuda()
print(decoder, flush=True)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
if USE_GPU:
  criterion.cuda()

start = time.time()
all_losses = []
loss_avg = 0

from tqdm import tqdm

for epoch in tqdm(range(1, n_epochs + 1)):
    loss = train(*random_training_set())       
    loss_avg += loss

    if epoch % print_every == 0:
        e = evaluate('Wh', 200)
        print('\n[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(e, '\n', flush=True)

    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

## Rysowanie wykresu funkcji straty

Wykreślanie historycznej straty z all_losses pokazuje uczenie sieci:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

plt.figure()
plt.plot(all_losses)

## Ocena w różnych "temperaturach"

W powyższej funkcji `evaluate`, za każdym razem, gdy dokonywana jest prognoza, wyjścia są dzielone przez przekazany argument "temperature". Używanie większej liczby sprawia, że wszystkie akcje są bardziej jednakowo prawdopodobne, a tym samym dają nam "bardziej losowe" ("more random") wyniki. Używanie mniejszej wartości (mniejszej niż 1) sprawia, że wysokie prawdopodobieństwa przyczyniają się bardziej. Gdy ustawiamy temperaturę na zero, wybieramy tylko najbardziej prawdopodobne wyjścia.

Możemy to zobaczyć poprzez dostosowanie argumentu `temperature`.


In [ ]:
print(evaluate('Th', 200, temperature=0.8))

[link text](https://)Lower temperatures are less varied, choosing only the more probable outputs:

In [ ]:
print(evaluate('Th', 200, temperature=0.2))

Higher temperatures more varied, choosing less probable outputs:

In [ ]:
print(evaluate('Th', 200, temperature=1.4))

## Ćwiczenia

* Trenuj z własnym zestawem danych, np.
     * Tekst od innego autora
     * Posty na blogu
     * Kod
* Zwiększ liczbę warstw i rozmiar sieci, aby uzyskać lepsze wyniki

**Następnie**: [Generowanie nazw z warunkowym poziomem RNN na poziomie znaku](https://github.com/spro/practical-pytorch/blob/master/conditional-char-rnn/conditional-char-rnn.ipynb)

## Zapisanie modelu

In [ ]:
ALLCHARS, MODEL = ['all_characters', 'model']

fn_pan_tadeusz = {ALLCHARS: 'all_characters.pan_tadeusz.p', MODEL: 'pan_tadeusz.h400.l3.3000.gpu.torch'}

fn_dict = fn_pan_tadeusz

In [ ]:
# save all_characters
import pickle

pickle.dump(all_characters, open(tmp_path / fn_dict[ALLCHARS], 'wb'))

In [ ]:
# all_characters = pickle.load( open( tmp_path / fn_dict[ALLCHARS], 'rb' ) )
# n_characters = len(all_characters)

In [ ]:
decoder.state_dict

In [ ]:
# save model
model_path = tmp_path / fn_dict[MODEL]
torch.save(decoder, model_path)

In [ ]:
 # decoder = torch.load(model_path)

In [ ]:
model_path

In [ ]:
ls -lah /content/data/rnn_generator/tmp/

## Monitorowanie maszyny wirtualnej

In [ ]:
import os
import psutil

def print_memsize():
  process = psutil.Process(os.getpid())
  print(f'{process.memory_info().rss / 1024**3:.5} GB')

In [ ]:
print_memsize()

In [ ]:
!uptime

In [ ]:
who